### Variational Quantum Eigensolver (VQE)?

Variational Quantum Eigensolver (VQE) is a hybrid quantum–classical algorithm for finding the ground-state energy of a Hamiltonian $H$. It works by:

1. Preparing a parameterized trial state $\ket{\psi(\boldsymbol\theta)}$ on a quantum processor.
2. Measuring the expectation value  
   \begin{equation*}
     E(\boldsymbol\theta) = \bra{\psi(\boldsymbol\theta)}\,H\,\ket{\psi(\boldsymbol\theta)}
   \end{equation*}
3. Using a classical optimizer to update $\boldsymbol\theta$ so as to minimize $E(\boldsymbol\theta)$.  
   By the variational principle, the minimal value of $E(\boldsymbol\theta)$ is an upper bound to the true ground-state energy.

For a hands-on walkthrough and code examples, see the [Qiskit VQE tutorial](https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver).  


In [1]:
from qiskit import QuantumCircuit
import numpy as np
from scipy.optimize import minimize
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit.circuit import Parameter

estimator = StatevectorEstimator()


### Cost Function

The VQE cost function computes the expectation value of the Hamiltonian $H$ with respect to the parameterized trial state $\ket{\psi(\boldsymbol\theta)}$:

\begin{equation*}
E(\boldsymbol\theta) \;=\; \langle \psi(\boldsymbol\theta)\,|\,H\,|\,\psi(\boldsymbol\theta)\rangle
\end{equation*}

In our implementation, the function `cost_func_vqe(parameters, ansatz, observable, estimator)`:

1. Binds `parameters` into the `ansatz` circuit $\ket{\psi(\boldsymbol\theta)}$.  
2. Uses the Qiskit `StatevectorEstimator` to evaluate $\langle H \rangle$.  
3. Returns the real part of the estimated energy, which the classical optimizer then minimizes.  


In [2]:
def cost_func_vqe(parameters, ansatz, observable, estimator):
    estimator_job = estimator.run([(ansatz, observable, [parameters])])
    estimator_result = estimator_job.result()[0]

    cost = estimator_result.data.evs[0]
    return cost


### Optimization Loop

The optimization loop is a classical routine that adjusts the variational parameters to minimize the measured energy. At each step, the circuit is prepared with the current parameters, the expectation value of the Hamiltonian is estimated, and the optimizer proposes a new parameter set. Gradient-free methods like COBYLA only require energy evaluations and handle noise well. 


In [3]:
def opti_loop_vqe(cost_func_vqe, x0, ansatz, observable, estimator, method="COBYLA"):
    return minimize(cost_func_vqe, x0, args=(ansatz, observable, estimator), method=method)


## Test Cases
In each case we:
1. Build or import an ansatz,
2. Define the Hamiltonian as a SparsePauliOp,
3. Initialize parameters,
4. Run `opti_loop_vqe` and print the minimal eigenvalue and final statevector.


### Test Case 1 — 2-Qubit Non-degenerate Hamiltonian

**Hamiltonian**  
\begin{equation*}
H = Z\otimes Z \;+\; 2\,X\otimes X
\end{equation*}

**Spectrum**  
-3, -1, +1, +3

**Expected**  
- **Ground energy**: -3  
- **Ground state**: $\displaystyle \ket{\psi_0} = \tfrac{1}{\sqrt2}(\ket{01} - \ket{10})$  
- VQE should converge to $\langle H\rangle \approx -3$ and prepare a state close to $\ket{\psi_0}$.



In [4]:
# 1. Build ansatz with two parameters
θ0, θ1 = Parameter('θ0'), Parameter('θ1')
ansatz1 = QuantumCircuit(2)
ansatz1.ry(θ0, 0)
ansatz1.ry(θ1, 1)
ansatz1.cx(0, 1)

# 2. Hamiltonian
obs1 = SparsePauliOp.from_list([
    ("ZZ", 1.0),
    ("XX", 2.0)
])

# 3. Initial guess
x0_1 = np.random.uniform(0, 2*np.pi, size=2)

# 4. Optimize
res1 = opti_loop_vqe(cost_func_vqe, x0_1, ansatz1, obs1, estimator)
print("Test 1 → min eigenvalue =", res1.fun)

# 5. Final statevector
param_dict = dict(zip(ansatz1.parameters, res1.x))
final_circ = ansatz1.assign_parameters(param_dict)
sv1 = Statevector(final_circ)
print("Final statevector:",
      " + ".join(f"({amp:.4f})|{idx:02b}⟩"
                 for idx, amp in enumerate(sv1.data) if abs(amp)>1e-3))


Test 1 → min eigenvalue = -2.99999998994355
Final statevector: (0.7071+0.0000j)|01⟩ + (-0.7071+0.0000j)|10⟩


### Test Case 2 — 3-Qubit Mixed-Interaction Hamiltonian using the TwoLocal ansatz

**Hamiltonian**  
\begin{equation*}
H = X\otimes X\otimes Z \;+\; Y\otimes Y\otimes Z \;+\; Z\otimes Z\otimes Z
\end{equation*}

**Spectrum**  
-3, -1, +1, +3

**Expected**  
- **Ground energy**: -3  
- **Ground state**: $\displaystyle \ket{\psi_0} = \tfrac{1}{\sqrt2}\bigl(\ket{010} - \ket{100}\bigr)$  
- VQE should converge to $\langle H\rangle \approx -3$ and prepare a state close to $\ket{\psi_0}$.


In [5]:
# 1. TwoLocal ansatz
ansatz2 = TwoLocal(num_qubits=3,
                   rotation_blocks=["rz","ry"],
                   entanglement_blocks="cx",
                   entanglement="linear",
                   reps=1)

# 2. Hamiltonian
obs2 = SparsePauliOp.from_list([
    ("XXZ", 1.0),
    ("YYZ", 1.0),
    ("ZZZ", 1.0)
])

# 3. Initial guess
x0_2 = np.random.uniform(0, 2*np.pi, size=len(ansatz2.parameters))

# 4. Optimize
res2 = opti_loop_vqe(cost_func_vqe, x0_2, ansatz2, obs2, estimator)
print("Test 2 → min eigenvalue =", res2.fun)

# 5. Final statevector
param_dict = dict(zip(ansatz2.parameters, res2.x))
sv2 = Statevector(ansatz2.assign_parameters(param_dict))

print("Final statevector:",
      " + ".join(f"({amp:.4f})|{idx:03b}⟩"
                 for idx, amp in enumerate(sv2.data) if abs(amp)>1e-3))


Test 2 → min eigenvalue = -2.9999999868878278
Final statevector: (-0.6731+0.2166j)|010⟩ + (0.6731-0.2166j)|100⟩


### Test case 3 - Random Hermitian matrix

In [6]:
def random_hermitian_matrix(n):
    dim = 2**n
    A = np.random.rand(dim, dim) + 1j*np.random.rand(dim, dim)
    H = (A + A.conj().T) / 2  # Make it Hermitian
    return H

n=3
H_matrix = random_hermitian_matrix(n)

Solve the diagonilization classicaly using scipy.linalg

In [7]:
from scipy.linalg import eigh

classical_eigvals, classical_eigvecs = eigh(H_matrix)
ground_energy_classical = classical_eigvals[0]
ground_state_classical = classical_eigvecs[:, 0]

print("Minimal eigenvalue found:", ground_energy_classical)


# Remove the global phase of a quantum statevector so that the first amplitude 
# is real and positive.
def fix_global_phase(vec: np.ndarray, threshold: float = 1e-3) -> np.ndarray:
    """
    Rotate the global phase so that the first amplitude with |amp|>threshold
    becomes real and positive.
    """
    v = vec.copy()
    # find first significant index
    for amp in v:
        if abs(amp) > threshold:
            phi = np.angle(amp)
            v *= np.exp(-1j * phi)
            if v[np.argmax(np.abs(v))].real < 0:
                v *= -1
            break
    return v

# Build the single-line sum
terms = []
for idx, amplitude in enumerate(fix_global_phase(ground_state_classical)):
    bstr = format(idx, f'0{n}b')
    terms.append(f"({amplitude:.5f})|{bstr}⟩")

# Join and print the sum
statevector_str = " + ".join(terms)
print("Final statevector:", statevector_str)

Minimal eigenvalue found: -0.9898308206354594
Final statevector: (0.23902-0.00000j)|000⟩ + (-0.32550+0.28952j)|001⟩ + (0.00129+0.10827j)|010⟩ + (0.52205+0.19213j)|011⟩ + (0.03754-0.48240j)|100⟩ + (-0.19626+0.12631j)|101⟩ + (-0.31924+0.03086j)|110⟩ + (0.06018-0.19198j)|111⟩


Find the ground state and the ground energy using VQE

In [8]:
# 1. TwoLocal ansatz
ansatz3 = TwoLocal(
    num_qubits=n,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=2
)

# 2. Hamiltonian in terms of Pauli operators
obs3 = SparsePauliOp.from_operator(Operator(H_matrix))

# 3. Initial guess
x0_3 = np.random.uniform(0, 2*np.pi, size=len(ansatz3.parameters))

# 4. Optimize
res3 = opti_loop_vqe(cost_func_vqe, x0_3, ansatz3, obs3, estimator)
print("Test 3 → min eigenvalue =", res3.fun)

# 5. Build final statevector and fix its global phase
param_dict3 = dict(zip(ansatz3.parameters, res3.x))
final_circ3 = ansatz3.assign_parameters(param_dict3)
sv3 = Statevector(final_circ3)

terms = []
for idx, amplitude in enumerate(fix_global_phase(sv3.data)):
    bstr = format(idx, '03b')
    terms.append(f"({amplitude:.5f})|{bstr}⟩")
statevector_str = " + ".join(terms)
print("Final statevector:", statevector_str)


Test 3 → min eigenvalue = -0.9873263657644158
Final statevector: (0.24538+0.00000j)|000⟩ + (-0.32126+0.27761j)|001⟩ + (-0.01020+0.08723j)|010⟩ + (0.49931+0.27033j)|011⟩ + (0.04750-0.48377j)|100⟩ + (-0.20420+0.10773j)|101⟩ + (-0.32814-0.02207j)|110⟩ + (0.07764-0.16005j)|111⟩


Comparison with the classical results

In [11]:
print("Classical ground energy: ", ground_energy_classical)
print("VQE ground energy: ", res3.fun)

# Compute and print the norm difference
diff = np.linalg.norm(fix_global_phase(ground_state_classical) - fix_global_phase(sv3.data))
print(f"\n||φ_c – φ_VQE|| = {diff:.6f}")


Classical ground energy:  -0.9898308206354594
VQE ground energy:  -0.9873263657644158

||φ_c – φ_VQE|| = 0.110102
